<a href="https://colab.research.google.com/github/InowaR/colab/blob/main/ParticleSimulator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import numpy as np
import time
import math

class Particle:
    def __init__(self, x1, y1, x2, y2, id):
        self.id = id
        self.position = [float(x1), float(y1)]
        self.velocity = [float(x2 - x1), float(y2 - y1)]
        self.prev_position = [float(x1), float(y1)]
        self.radius = 0.5

    def move(self):
        """Перемещает частицу согласно ее скорости"""
        self.prev_position = self.position.copy()
        self.position[0] += self.velocity[0]
        self.position[1] += self.velocity[1]

    def get_display_cell(self):
        """Возвращает координаты для отображения"""
        x = int(round(self.position[0]))
        y = int(round(self.position[1]))
        return (x, y)

    def reverse_velocity_x(self):
        self.velocity[0] = -self.velocity[0]

    def reverse_velocity_y(self):
        self.velocity[1] = -self.velocity[1]

    def distance_to(self, other_particle):
        """Вычисляет расстояние до другой частицы"""
        dx = self.position[0] - other_particle.position[0]
        dy = self.position[1] - other_particle.position[1]
        return math.sqrt(dx*dx + dy*dy)

class ParticleSimulator:
    def __init__(self, size=10):
        self.size = size
        self.matrix = np.zeros((size, size), dtype=int)
        self.particles = []

    def add_particle(self, x1, y1, x2, y2):
        """Добавляет частицу в симуляцию"""
        particle_id = len(self.particles) + 1
        particle = Particle(x1, y1, x2, y2, particle_id)
        self.particles.append(particle)
        return particle

    def update_matrix(self):
        """Обновляет матрицу с гарантированным сохранением индексов"""
        self.matrix = np.zeros((self.size, self.size), dtype=int)

        # Сначала обновляем все позиции частиц
        occupied_cells = {}
        for particle in self.particles:
            x, y = particle.get_display_cell()
            if 0 <= x < self.size and 0 <= y < self.size:
                occupied_cells[(x, y)] = particle.id

        # Если есть конфликты (несколько частиц в одной клетке),
        # используем приоритет по ID или последнюю обработанную
        for (x, y), particle_id in occupied_cells.items():
            self.matrix[y, x] = particle_id

    def check_wall_collisions(self):
        """Проверяет столкновения со стенами"""
        for particle in self.particles:
            x, y = particle.position

            # Столкновение с левой или правой стенкой
            if x < 0:
                particle.reverse_velocity_x()
                particle.position[0] = 0.1
            elif x >= self.size:
                particle.reverse_velocity_x()
                particle.position[0] = self.size - 1.1

            # Столкновение с верхней или нижней стенкой
            if y < 0:
                particle.reverse_velocity_y()
                particle.position[1] = 0.1
            elif y >= self.size:
                particle.reverse_velocity_y()
                particle.position[1] = self.size - 1.1

    def check_particle_collisions(self):
        """Проверяет и обрабатывает столкновения между частицами"""
        n = len(self.particles)
        collision_occurred = False

        # Создаем копии скоростей для одновременного обновления
        new_velocities = [particle.velocity.copy() for particle in self.particles]
        need_separation = [False] * n
        separation_vectors = [None] * n

        for i in range(n):
            for j in range(i + 1, n):
                p1 = self.particles[i]
                p2 = self.particles[j]

                # Проверяем столкновение
                distance = p1.distance_to(p2)
                if distance <= p1.radius + p2.radius:
                    collision_occurred = True
                    self.calculate_collision(i, j, new_velocities, need_separation, separation_vectors)

        # Применяем новые скорости и разделения
        for i in range(n):
            self.particles[i].velocity = new_velocities[i]
            if need_separation[i] and separation_vectors[i]:
                self.particles[i].position[0] += separation_vectors[i][0]
                self.particles[i].position[1] += separation_vectors[i][1]

        return collision_occurred

    def calculate_collision(self, i, j, new_velocities, need_separation, separation_vectors):
        """Вычисляет результат столкновения между двумя частицами"""
        p1 = self.particles[i]
        p2 = self.particles[j]

        # Вектор от p1 к p2
        dx = p2.position[0] - p1.position[0]
        dy = p2.position[1] - p1.position[1]

        # Нормализуем вектор столкновения
        distance = max(math.sqrt(dx*dx + dy*dy), 0.001)
        collision_vector = [dx/distance, dy/distance]

        # Текущие скорости
        v1 = new_velocities[i]
        v2 = new_velocities[j]

        # Вычисляем относительную скорость
        relative_velocity = [v1[0] - v2[0], v1[1] - v2[1]]

        # Скалярное произведение относительной скорости и вектора столкновения
        speed_along_collision = (relative_velocity[0] * collision_vector[0] +
                               relative_velocity[1] * collision_vector[1])

        # Если частицы уже удаляются друг от друга, не обрабатываем столкновение
        if speed_along_collision < 0:
            return

        # Импульс вдоль вектора столкновения
        impulse = speed_along_collision

        # Обновляем скорости (обмен проекциями скоростей вдоль вектора столкновения)
        new_velocities[i][0] -= impulse * collision_vector[0]
        new_velocities[i][1] -= impulse * collision_vector[1]
        new_velocities[j][0] += impulse * collision_vector[0]
        new_velocities[j][1] += impulse * collision_vector[1]

        # Разделяем частицы, чтобы избежать залипания
        separation = 0.8
        move_x = separation * collision_vector[0]
        move_y = separation * collision_vector[1]

        need_separation[i] = True
        need_separation[j] = True
        separation_vectors[i] = [-move_x, -move_y]
        separation_vectors[j] = [move_x, move_y]

        print(f"СТОЛКНОВЕНИЕ! Частицы {p1.id} и {p2.id} обменялись импульсами")

    def check_all_collisions(self):
        """Проверяет все типы столкновений"""
        self.check_wall_collisions()
        return self.check_particle_collisions()

    def simulate_step(self):
        """Выполняет один шаг симуляции"""
        for particle in self.particles:
            particle.move()

        collision_occurred = self.check_all_collisions()
        self.update_matrix()
        return collision_occurred

    def print_matrix(self):
        """Выводит матрицу в консоль"""
        print("+" + "-" * (self.size * 2 + 1) + "+")
        for row in self.matrix:
            print("|", end=" ")
            for cell in row:
                if cell == 0:
                    print("·", end=" ")
                else:
                    print(cell, end=" ")
            print("|")
        print("+" + "-" * (self.size * 2 + 1) + "+")

    def print_particle_info(self):
        """Выводит информацию о всех частицах"""
        for particle in self.particles:
            display_x, display_y = particle.get_display_cell()
            print(f"Частица {particle.id}: позиция ({particle.position[0]:.1f}, {particle.position[1]:.1f}), "
                  f"скорость ({particle.velocity[0]:.1f}, {particle.velocity[1]:.1f})")

    def run_simulation(self, steps=50, delay=0.5):
        """Запускает симуляцию на указанное количество шагов"""
        print("=" * 50)
        print("СИМУЛЯЦИЯ СТОЛКНОВЕНИЙ ЧАСТИЦ")
        print("=" * 50)
        print("Начальное состояние:")
        self.update_matrix()
        self.print_matrix()
        self.print_particle_info()

        for step in range(steps):
            print(f"\nШаг {step + 1}:")
            collision_occurred = self.simulate_step()
            self.print_matrix()
            self.print_particle_info()

            if collision_occurred:
                print(">>> Произошло столкновение частиц! <<<")

            time.sleep(delay)

# Демонстрационные сценарии
def demo_head_on_collision():
    """Демонстрация лобового столкновения"""
    print("\n" + "="*60)
    print("ДЕМО: Лобовое столкновение")
    print("="*60)

    simulator = ParticleSimulator(10)

    simulator.add_particle(2, 5, 3, 5)  # Движется вправо
    simulator.add_particle(7, 5, 6, 5)  # Движется влево

    simulator.run_simulation(steps=12, delay=0.4)

def demo_diagonal_collision():
    """Демонстрация диагонального столкновения"""
    print("\n" + "="*60)
    print("ДЕМО: Диагональное столкновение")
    print("="*60)

    simulator = ParticleSimulator(10)

    simulator.add_particle(2, 2, 3, 3)  # Движется вправо-вниз
    simulator.add_particle(7, 7, 6, 6)  # Движется влево-вверх

    simulator.run_simulation(steps=15, delay=0.3)

def demo_glancing_collision():
    """Демонстрация касательного столкновения"""
    print("\n" + "="*60)
    print("ДЕМО: Касательное столкновение")
    print("="*60)

    simulator = ParticleSimulator(10)

    simulator.add_particle(3, 2, 4, 3)  # Движется вправо-вниз
    simulator.add_particle(2, 4, 3, 3)  # Движется вправо-вверх

    simulator.run_simulation(steps=10, delay=0.4)

def demo_three_particles_collision():
    """Демонстрация столкновения трех частиц"""
    print("\n" + "="*60)
    print("ДЕМО: Столкновение трех частиц")
    print("="*60)

    simulator = ParticleSimulator(12)

    simulator.add_particle(3, 3, 4, 3)   # Движется вправо
    simulator.add_particle(8, 3, 7, 3)   # Движется влево
    simulator.add_particle(5, 6, 5, 5)   # Движется вверх

    simulator.run_simulation(steps=15, delay=0.3)

def demo_perpendicular_collision():
    """Демонстрация перпендикулярного столкновения"""
    print("\n" + "="*60)
    print("ДЕМО: Перпендикулярное столкновение")
    print("="*60)

    simulator = ParticleSimulator(10)

    simulator.add_particle(2, 5, 3, 5)  # Движется вправо (горизонтально)
    simulator.add_particle(5, 2, 5, 3)  # Движется вниз (вертикально)

    simulator.run_simulation(steps=10, delay=0.4)

if __name__ == "__main__":
    # Запускаем различные демонстрационные сценарии
    demo_head_on_collision()
    demo_diagonal_collision()
    demo_glancing_collision()
    demo_three_particles_collision()
    demo_perpendicular_collision()


ДЕМО: Лобовое столкновение
СИМУЛЯЦИЯ СТОЛКНОВЕНИЙ ЧАСТИЦ
Начальное состояние:
+---------------------+
| · · · · · · · · · · |
| · · · · · · · · · · |
| · · · · · · · · · · |
| · · · · · · · · · · |
| · · · · · · · · · · |
| · · 1 · · · · 2 · · |
| · · · · · · · · · · |
| · · · · · · · · · · |
| · · · · · · · · · · |
| · · · · · · · · · · |
+---------------------+
Частица 1: позиция (2.0, 5.0), скорость (1.0, 0.0)
Частица 2: позиция (7.0, 5.0), скорость (-1.0, 0.0)

Шаг 1:
+---------------------+
| · · · · · · · · · · |
| · · · · · · · · · · |
| · · · · · · · · · · |
| · · · · · · · · · · |
| · · · · · · · · · · |
| · · · 1 · · 2 · · · |
| · · · · · · · · · · |
| · · · · · · · · · · |
| · · · · · · · · · · |
| · · · · · · · · · · |
+---------------------+
Частица 1: позиция (3.0, 5.0), скорость (1.0, 0.0)
Частица 2: позиция (6.0, 5.0), скорость (-1.0, 0.0)

Шаг 2:
СТОЛКНОВЕНИЕ! Частицы 1 и 2 обменялись импульсами
+---------------------+
| · · · · · · · · · · |
| · · · · · · · · · · |
|